# Analisis de prestaciones por diagnostico

- En este apartado sequiere ver la cantidad de prestaciones que se realizan por cada diagnostico principal.

- El resultado que se quiere obtener es parecido al siguiente:

| Año  | Diagnostico | Resumen Procedimientos     |
| ---- | ----------- | -------------------------- |
| 2019 | "I25.1"     | {"45.23": 20, "73.59": 10} |
| 2020 | "I25.1"     | {"45.23": 7, "73.59": 3}   |


In [53]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_DEL_TORAX = 112103


In [55]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";")

In [56]:
procedimientos_long = pd.melt(
    df_procesada,
    id_vars=["COD_HOSPITAL", "ANIO_EGRESO", "DIAGNOSTICO1"],
    value_vars=[f"PROCEDIMIENTO{i}" for i in range(1, 31)],
)

In [57]:
conteo_procedimientos = procedimientos_long.groupby(["ANIO_EGRESO", "COD_HOSPITAL"])[
    "value"
].value_counts()

In [58]:
conteo_procedimientos = conteo_procedimientos.reset_index(name="conteo")

In [59]:
conteo_procedimientos = conteo_procedimientos.sort_values(
    ["ANIO_EGRESO", "value", "conteo"], ascending=False
)


In [70]:
conteo_procedimientos["ranking"] = (
    conteo_procedimientos.groupby(["ANIO_EGRESO", "value"]).cumcount() + 1
)

conteo_procedimientos["diferencia_proximo_hospital"] = conteo_procedimientos["conteo"].diff(
    periods=-1
)

conteo_procedimientos["diferencia_proximo_hospital"] = np.where(
    conteo_procedimientos["diferencia_proximo_hospital"] < 0,
    0,
    conteo_procedimientos["diferencia_proximo_hospital"],
)

In [71]:
mejores_procedimientos_torax = conteo_procedimientos.query(
    "COD_HOSPITAL == @HOSPITAL_DEL_TORAX and ranking == 1"
)

In [74]:
mejores_procedimientos_torax

,ANIO_EGRESO,COD_HOSPITAL,value,conteo,ranking,diferencia_proximo_hospital
177062,2021.0,112103,93.91,124,1,36.0
177508,2021.0,112103,93.76,1,1,0.0
177502,2021.0,112103,91.76,1,1,0.0
177110,2021.0,112103,89.49,52,1,19.0
177034,2021.0,112103,89.48,277,1,81.0
...,...,...,...,...,...,...
35062,2019.0,112103,7.16,10,1,9.0
34826,2019.0,112103,0.66,831,1,40.0
34905,2019.0,112103,0.59,102,1,65.0
35087,2019.0,112103,0.52,7,1,1.0
